# Project Setup

This notebook will guide you through setting up the project environment for using CrewAI. We will:

1. Install the required Python modules.
2. Set up a virtual environment.
3. Verify the installation.

In [ ]:
# Step 1: Create and activate a virtual environment
%python3 -m venv venv
%source venv/bin/activate

# Step 2: Install required Python modules
%pip install -r requirements.txt

# Step 3: Verify installation
%pip list

# Setting Up OpenAI API Key

To use CrewAI with the OpenAI API, you need to set your OpenAI API Key as an environment variable. This key is required for CrewAI to interact with OpenAI's services. Replace `your_openai_api_key` with your actual API key in the code block below.

In [ ]:
# Step 4: Set OpenAI API Key from .env file
import os
from dotenv import load_dotenv
from IPython.display import Markdown
from crewai import LLM, Agent, Task, Crew

# Load environment variables from .env file
load_dotenv()

# Uncomment the code block below to use OpenAI with your API Key
api_key = os.getenv('OPENAI_API_KEY')
if not api_key:
    raise ValueError("OPENAI_API_KEY is not set in the .env file")

llm = LLM(
    model="gpt-4o",  # Specify the OpenAI model you want to use
    api_key=api_key
)

# Uncomment the code block below to use Ollama with your local model
# Make sure to have Ollama installed and running
# 
# llm = LLM(
#     model="ollama/llama3.2:1b",
#     base_url="http://localhost:11434"
# )



# Define Agents
This block defines multiple agents with specific roles, goals, and backstories. Each agent is configured to use the LLM defined earlier.

In [12]:
senior_technical_writer = Agent(
    role="Senior Technical Writer",
    
    goal="""Craft clear, engaging, and well-structured
            technical content based on research findings""",
    
    backstory="""You are an experienced technical writer
                with expertise in simplifying complex
                concepts, structuring content for readability,
                and ensuring accuracy in documentation.""",
                
    llm=llm,
                
    verbose=True
)

research_analyst = Agent(
    role="Senior Research Analyst",
    goal="""Find, analyze, and summarize information 
            from various sources to support technical 
            and business-related inquiries.""",
    backstory="""You are a skilled research analyst with expertise 
                in gathering accurate data, identifying key trends, 
                and presenting insights in a structured manner.""",
    llm=llm,
    verbose=True
)

code_reviewer = Agent(
    role="Senior Code Reviewer",
    goal="""Review code for bugs, inefficiencies, and 
            security vulnerabilities while ensuring adherence 
            to best coding practices.""",
    backstory="""You are a seasoned software engineer with years of 
                experience in writing, reviewing, and optimizing 
                production-level code in multiple programming languages.""",
    llm=llm,
    verbose=True
)

legal_reviewer = Agent(
    role="Legal Document Expert Reviewer",
    goal="""Review contracts and legal documents to 
            ensure compliance with applicable laws and 
            highlight potential risks.""",
    backstory="""You are a legal expert with deep knowledge 
                of contract law, regulatory frameworks, 
                and risk mitigation strategies.""",
    llm=llm,
    verbose=True
)


# Define a Writing Task
This block defines a writing task for the Senior Technical Writer agent. The task includes a description, the agent responsible, and the expected output.

In [ ]:

writing_task = Task(
    description="""Write a well-structured, engaging,
                   and technically accurate article
                   on {topic}.""",
    
    agent=senior_technical_writer, 
    
    
    expected_output="""A polished, detailed, and easy-to-read
                       article on the given topic.""",
)


# Create a Crew and Execute the Task
This block creates a crew to manage the task and agents. It then kicks off the crew with the specified input and displays the response.

In [ ]:
crew = Crew(
    agents=[senior_technical_writer],
    tasks=[writing_task],
    verbose=True
)

response = crew.kickoff(inputs={"topic":"AI Agents"})

# Display the response in Markdown format
# Markdown(response.raw)

# Save the Response to a File
This block saves the raw response from the crew execution to a Markdown file for further use or documentation.

In [ ]:
f = open("build/output.md", "w")
f.write(response.raw)
f.close()

# Define a file reader tool

In [15]:
from crewai_tools import FileReadTool

file_read_tool = FileReadTool()